# **SCRIPT PREPARAZIONE CARTELLE PER IL TRAINING**

Questo script ha lo scopo di creare in maniera totalmente automatica tutto il necessario per il training di **YOLO Pose**. In particolare organizza le cartelle in questa disposizionei:

*   TRAIN (70%): 210
*   VALIDATION (20%): 60
*   TEST (10%): 30



```
.
└── Training_Dataset/
    ├── train/
    │   ├── images
    │   └── labels
    ├── validation/
    │   ├── images
    │   └── labels
    └── test/
        ├── images
        └── labels
```





---



In [ ]:
# === MOUNT DRIVE ===
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import shutil

# === Percorsi ===
base_input = "/content/drive/MyDrive/Computer_Vision_&_Deep_Learning/DATASET/Datasets_PCTO/Dataset_Annotated_PCTO"
images_input = os.path.join(base_input, "images")
labels_input = os.path.join(base_input, "labels_corrette")

base_output = "/content/drive/MyDrive/Computer_Vision_&_Deep_Learning/DATASET/Datasets_PCTO/Dataset_Annotated_PCTO_702010"
splits = {
    "train": (0, 210),        # 70%
    "validation": (210, 270), # 20%
    "test": (270, 300)        # 10%
}

# === Crea le directory ===
for split in splits:
    os.makedirs(os.path.join(base_output, split, "images"), exist_ok=True)
    os.makedirs(os.path.join(base_output, split, "labels"), exist_ok=True)

# === Elenco immagini ordinato ===
all_images = sorted([
    f for f in os.listdir(images_input)
    if f.lower().endswith((".jpg", ".jpeg", ".png"))
])

# === Funzione per copiare immagini e labels ===
def copy_split(start, end, split_name):
    for img_file in all_images[start:end]:
        base_name = os.path.splitext(img_file)[0]
        label_file = base_name + ".txt"

        src_img_path = os.path.join(images_input, img_file)
        src_lbl_path = os.path.join(labels_input, label_file)

        dst_img_path = os.path.join(base_output, split_name, "images", img_file)
        dst_lbl_path = os.path.join(base_output, split_name, "labels", label_file)

        if os.path.exists(src_lbl_path):
            shutil.copy(src_img_path, dst_img_path)
            shutil.copy(src_lbl_path, dst_lbl_path)

# === Esegui la suddivisione ===
for split_name, (start, end) in splits.items():
    copy_split(start, end, split_name)

print("✅ Dataset suddiviso in 70% train, 20% validation, 10% test con immagini e label salvate correttamente.")


✅ Dataset suddiviso in 70% train, 20% validation, 10% test con immagini e label salvate correttamente.


ALTRA VERSIONE PER FARE LA DIVISIONE DEI SET (CON SHUFFLE E PERCENTUALI)

In [ ]:
import os
import shutil
import random

# === Percorsi ===
base_input = "/content/drive/MyDrive/Computer_Vision_&_Deep_Learning/DATASET/Datasets_PCTO/Dataset_Annotated_PCTO"
images_input = os.path.join(base_input, "images")
labels_input = os.path.join(base_input, "labels_corrette")

base_output = "/content/drive/MyDrive/Computer_Vision_&_Deep_Learning/DATASET/Datasets_PCTO/Dataset_Annotated_PCTO_702010"

# === Elenco immagini ordinato e con label esistente ===
all_images = sorted([
    f for f in os.listdir(images_input)
    if f.lower().endswith((".jpg", ".jpeg", ".png")) and
    os.path.exists(os.path.join(labels_input, os.path.splitext(f)[0] + ".txt"))
])

# === Shuffle ===
random.seed(42)
random.shuffle(all_images)

# === Split dinamico ===
total = len(all_images)
train_split = int(0.7 * total)
val_split = int(0.9 * total)

splits = {
    "train": all_images[:train_split],
    "validation": all_images[train_split:val_split],
    "test": all_images[val_split:]
}

# === Crea le directory ===
for split in splits:
    os.makedirs(os.path.join(base_output, split, "images"), exist_ok=True)
    os.makedirs(os.path.join(base_output, split, "labels"), exist_ok=True)

# === Copia file ===
for split_name, files in splits.items():
    for img_file in files:
        base_name = os.path.splitext(img_file)[0]
        label_file = base_name + ".txt"

        src_img_path = os.path.join(images_input, img_file)
        src_lbl_path = os.path.join(labels_input, label_file)

        dst_img_path = os.path.join(base_output, split_name, "images", img_file)
        dst_lbl_path = os.path.join(base_output, split_name, "labels", label_file)

        shutil.copy(src_img_path, dst_img_path)
        shutil.copy(src_lbl_path, dst_lbl_path)

    print(f"✅ {split_name.capitalize()}: {len(files)} immagini copiate")

print("\n✔️ Dataset suddiviso in train (70%), validation (20%), test (10%) con shuffle e controllo label.")




---



# **SCRIPT DATA AUGMENTATION E DIRECTORY TRAINING**

Prende le 210 immagini di training dalla directory Dataset_Annotated_PCTO_702010/train/.

Per ognuna:

* la copia nella nuova directory Training_Test_BadAnnotation_YesAugmentation/images/train/

* crea 2 versioni augmentate applicando:

  * rotazione ±10°

  * flip orizzontale

  * variazioni di luminosità/contrasto

  * sfocatura gaussiana

aggiorna e salva i corrispondenti file .txt delle annotazioni keypoints.

Copia anche la parte validation già esistente nel posto giusto.

Mantiene 15 keypoints per ogni bbox (con padding se necessario).

In [ ]:
!pip install -q albumentations

In [ ]:
import os
import cv2
import shutil
import albumentations as A

# === Percorsi ===
base_input = "/content/drive/MyDrive/Computer_Vision_&_Deep_Learning/DATASET/Datasets_PCTO/Dataset_Annotated_PCTO_702010"
train_img_input = os.path.join(base_input, "train", "images")
train_lbl_input = os.path.join(base_input, "train", "labels")

base_output = "/content/drive/MyDrive/Computer_Vision_&_Deep_Learning/DATASET/Training_Test_Datasets/Training_Test_BadAnnotation_YesAugmentation"
train_img_out = os.path.join(base_output, "images", "train")
train_lbl_out = os.path.join(base_output, "labels", "train")

# === Crea cartelle output se non esistono ===
os.makedirs(train_img_out, exist_ok=True)
os.makedirs(train_lbl_out, exist_ok=True)

# === Copia val così com'è (solo se non già fatto) ===
val_img_src = os.path.join(base_input, "validation", "images")
val_lbl_src = os.path.join(base_input, "validation", "labels")
val_img_dst = os.path.join(base_output, "images", "val")
val_lbl_dst = os.path.join(base_output, "labels", "val")
os.makedirs(val_img_dst, exist_ok=True)
os.makedirs(val_lbl_dst, exist_ok=True)
for src_folder, dst_folder in [(val_img_src, val_img_dst), (val_lbl_src, val_lbl_dst)]:
    for file in os.listdir(src_folder):
        shutil.copy(os.path.join(src_folder, file), os.path.join(dst_folder, file))

# === Definizione della trasformazione ===
transform = A.Compose([
    A.Rotate(limit=10, p=0.5),
    A.HorizontalFlip(p=0.3),
    A.RandomBrightnessContrast(p=0.4),
    A.GaussianBlur(p=0.2)
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

# === Funzione di conversione e augmentazione ===
def augment_and_save(image_path, label_path, output_img_dir, output_lbl_dir):
    image = cv2.imread(image_path)
    h, w = image.shape[:2]
    base_name = os.path.splitext(os.path.basename(image_path))[0]

    with open(label_path, 'r') as f:
        lines = f.readlines()

    # Parse ogni oggetto
    objects = []
    for line in lines:
        parts = line.strip().split()
        class_id = int(parts[0])
        xc, yc, bw, bh = map(float, parts[1:5])
        keypoints = []
        for i in range(5, len(parts), 3):
            xk, yk, v = float(parts[i]), float(parts[i+1]), int(parts[i+2])
            keypoints.append((xk, yk, v))
        objects.append((class_id, [xc, yc, bw, bh], keypoints))

    for aug_idx in range(1, 3):  # 2 immagini aumentate
        bboxes = [obj[1] for obj in objects]
        keypoints_all = [kp for obj in objects for kp in obj[2]]
        visibility = [kp[2] for kp in keypoints_all]
        visible_kp = [(kp[0], kp[1]) for kp in keypoints_all if kp[2] > 0]

        try:
            augmented = transform(image=image, bboxes=bboxes, keypoints=visible_kp)
        except Exception as e:
            print(f"❌ Errore su {image_path}: {e}")
            continue

        aug_image = augmented['image']
        aug_bboxes = augmented['bboxes']
        aug_keypoints = augmented['keypoints']

        new_lines = []
        idx_kp = 0
        for j, (class_id, _, _) in enumerate(objects):
            x_c, y_c, bw, bh = aug_bboxes[j]
            line = [str(class_id), str(x_c), str(y_c), str(bw), str(bh)]
            for _ in range(15):  # 15 keypoint attesi
                if idx_kp < len(aug_keypoints):
                    xk, yk = aug_keypoints[idx_kp]
                    vk = visibility[idx_kp]
                    xk = min(max(xk, 0.0), 1.0)
                    yk = min(max(yk, 0.0), 1.0)
                    line.extend([str(xk), str(yk), str(vk)])
                    idx_kp += 1
                else:
                    line.extend(['0.0', '0.0', '0'])
            new_lines.append(' '.join(line))

        # Salvataggio immagini e labels
        new_img_name = f"{base_name}_aug{aug_idx}.jpg"
        new_lbl_name = f"{base_name}_aug{aug_idx}.txt"

        cv2.imwrite(os.path.join(output_img_dir, new_img_name), aug_image)
        with open(os.path.join(output_lbl_dir, new_lbl_name), 'w') as f:
            f.write('\n'.join(new_lines))

# === Esegui su tutto il train ===
for img_file in sorted(os.listdir(train_img_input)):
    if not img_file.lower().endswith((".jpg", ".jpeg", ".png")):
        continue
    base_name = os.path.splitext(img_file)[0]
    img_path = os.path.join(train_img_input, img_file)
    lbl_path = os.path.join(train_lbl_input, base_name + ".txt")

    # Copia originali
    shutil.copy(img_path, os.path.join(train_img_out, img_file))
    shutil.copy(lbl_path, os.path.join(train_lbl_out, base_name + ".txt"))

    # Augmenta e salva
    augment_and_save(img_path, lbl_path, train_img_out, train_lbl_out)

"✅ Augmentazione completata. Tutti i dati salvati in Training_Test_BadAnnotation_YesAugmentation/"
